# Niveau 3 Corrigé - Test des Améliorations

Ce notebook teste les corrections apportées au Level 3:
- Configuration assouplies (seuils d'acceptation)
- Mapping amélioré articles → source_id
- Critères de re-summarize moins restrictifs
- Nouveaux garde-fous pour tier CRITICAL

In [ ]:
import sys, os, json, pandas as pd
from pathlib import Path
import logging

# Setup paths
def find_project_root():
    p = Path.cwd().resolve()
    for parent in [p, *p.parents]:
        if (parent / "src").exists() and (parent / "outputs").exists():
            return parent
    return Path.cwd()

PROJECT_ROOT = find_project_root()
sys.path.append(str(PROJECT_ROOT / "src" / "detection" / "level3_improvement"))

print(f"Project root: {PROJECT_ROOT}")

In [ ]:
# Import Level3 utils with corrections
from level3_utils import (
    sha1_text, read_jsonl, write_jsonl, detect_lang, 
    choose_mode, accept_after, l2_like_evaluate
)
import yaml

# Load corrected config
config_file = PROJECT_ROOT / "src" / "detection" / "level3_improvement" / "config" / "level3.yaml"
with open(config_file, "r", encoding="utf-8") as f:
    CFG = yaml.safe_load(f)

print("Configuration corrigée chargée:")
print(f"- priority_threshold: {CFG['priority_threshold']}")
print(f"- min_text_chars_for_resummarize: {CFG['min_text_chars_for_resummarize']}")
print(f"- accepted_tiers: {CFG['acceptance']['accepted_tiers']}")
print(f"- topic overlap min: {CFG.get('acceptance_topic', {}).get('after_text_min', 'N/A')}")

In [ ]:
# Test du mapping amélioré
outputs_dir = PROJECT_ROOT / "outputs"

# Charger le mapping amélioré
enhanced_mapping = pd.read_csv(outputs_dir / "level3_enhanced_mapping.csv")
text_mapping = pd.read_csv(outputs_dir / "level3_text_mapping.csv")

print(f"Mapping amélioré: {len(enhanced_mapping)} entrées")
print(f"Avec texte: {enhanced_mapping['has_text'].sum()} ({enhanced_mapping['has_text'].mean()*100:.1f}%)")
print(f"Texte suffisant: {enhanced_mapping['enough_length'].sum()} ({enhanced_mapping['enough_length'].mean()*100:.1f}%)")

print(f"\nMapping texte pour Level3: {len(text_mapping)} entrées")
print(f"Longueur moyenne des textes: {text_mapping['text_length'].mean():.0f} caractères")

In [ ]:
# Test de la sélection des candidats avec nouveaux seuils
level2_results = pd.read_csv(outputs_dir / "level2_simplified_results_with_ids.csv")

# Sélection avec seuil abaissé
candidates_new = level2_results[
    (level2_results["tier"] == "CRITICAL") | 
    (level2_results["level3_priority_final"] >= CFG["priority_threshold"])
].copy()

print(f"Candidats avec nouveaux seuils: {len(candidates_new)}")
print(f"Distribution par tier:")
print(candidates_new["tier"].value_counts())
print(f"\nDistribution par stratégie:")
print(candidates_new["strategy"].value_counts() if "strategy" in candidates_new.columns else "Stratégie non trouvée")

In [ ]:
# Test de choose_mode avec critères assouplies
sample_candidates = candidates_new.head(20).copy()

# Ajouter les informations de texte depuis le mapping amélioré
sample_with_text = sample_candidates.merge(
    enhanced_mapping[['level2_id', 'has_text', 'enough_length', 'text_length', 'text']], 
    left_on='summary_id', 
    right_on='level2_id', 
    how='left'
)

# Test des modes
mode_results = []
for _, row in sample_with_text.iterrows():
    mode, reason, flags = choose_mode(row.to_dict(), CFG)
    mode_results.append({
        'summary_id': row['summary_id'],
        'tier': row['tier'],
        'strategy': row.get('strategy', 'unknown'),
        'has_text': flags['has_text'],
        'enough_length': flags['enough_length'],
        'text_length': row.get('text_length', 0),
        'mode': mode,
        'reason': reason
    })

mode_df = pd.DataFrame(mode_results)
print("Test des modes avec critères assouplies:")
print(mode_df[['summary_id', 'tier', 'strategy', 'has_text', 'enough_length', 'mode', 'reason']])
print(f"\nDistribution des modes:")
print(mode_df['mode'].value_counts())

In [ ]:
# Test des critères d'acceptation assouplies
print("Test des nouveaux critères d'acceptation:")

# Simuler des cas avant/après pour tester accept_after
test_cases = [
    {
        'name': 'CRITICAL avec amélioration factualité',
        'before': {'tier': 'CRITICAL', 'factuality_score': 0.65, 'coherence_score': 0.70},
        'after': {'tier': 'CRITICAL', 'factuality_score': 0.82, 'coherence_score': 0.72, 'issues_count': 5}
    },
    {
        'name': 'MODERATE assouplies',
        'before': {'tier': 'CRITICAL', 'factuality_score': 0.70, 'coherence_score': 0.65},
        'after': {'tier': 'MODERATE', 'factuality_score': 0.82, 'coherence_score': 0.72, 'issues_count': 3}
    },
    {
        'name': 'GOOD (accepté)',
        'before': {'tier': 'CRITICAL', 'factuality_score': 0.70, 'coherence_score': 0.65},
        'after': {'tier': 'GOOD', 'factuality_score': 0.85, 'coherence_score': 0.75, 'issues_count': 2}
    }
]

for test_case in test_cases:
    accepted, reason = accept_after(test_case['before'], test_case['after'], CFG)
    print(f"\n{test_case['name']}: {'✅ ACCEPTÉ' if accepted else '❌ REJETÉ'} - {reason}")
    print(f"  Avant: {test_case['before']}")
    print(f"  Après: {test_case['after']}")

In [ ]:
# Validation finale: comparaison ancien vs nouveau système
print("=== COMPARAISON ANCIEN vs NOUVEAU SYSTÈME ===")

# Anciens seuils (pour comparaison)
old_priority_threshold = 0.85
old_min_chars = 800
old_topic_threshold = 0.12

# Nouveaux seuils
new_priority_threshold = CFG['priority_threshold']
new_min_chars = CFG['min_text_chars_for_resummarize']
new_topic_threshold = CFG.get('acceptance_topic', {}).get('after_text_min', 0.05)

print(f"Seuil priorité: {old_priority_threshold} → {new_priority_threshold}")
print(f"Caractères min: {old_min_chars} → {new_min_chars}")
print(f"Topic overlap: {old_topic_threshold} → {new_topic_threshold}")

# Impact sur la sélection
old_candidates = level2_results[
    (level2_results["tier"] == "CRITICAL") | 
    (level2_results["level3_priority_final"] >= old_priority_threshold)
]

print(f"\nCandidats sélectionnés:")
print(f"  Ancien système: {len(old_candidates)}")
print(f"  Nouveau système: {len(candidates_new)}")
print(f"  Gain: +{len(candidates_new) - len(old_candidates)} candidats")

# Impact sur les textes suffisants
old_sufficient = enhanced_mapping[enhanced_mapping['text_length'] >= old_min_chars]
new_sufficient = enhanced_mapping[enhanced_mapping['text_length'] >= new_min_chars]

print(f"\nTextes suffisants pour re-summarize:")
print(f"  Ancien système: {len(old_sufficient)} ({len(old_sufficient)/len(enhanced_mapping)*100:.1f}%)")
print(f"  Nouveau système: {len(new_sufficient)} ({len(new_sufficient)/len(enhanced_mapping)*100:.1f}%)")
print(f"  Gain: +{len(new_sufficient) - len(old_sufficient)} textes utilisables")

In [ ]:
# Résumé des améliorations
print("\n" + "="*50)
print("RÉSUMÉ DES CORRECTIONS LEVEL 3")
print("="*50)

print("\n✅ CORRECTIONS APPLIQUÉES:")
print("1. Configuration assouplies:")
print(f"   - Seuil priorité: 0.85 → {CFG['priority_threshold']}")
print(f"   - Texte min: 800 → {CFG['min_text_chars_for_resummarize']} caractères")
print(f"   - Tiers acceptés: {CFG['acceptance']['accepted_tiers']}")
print(f"   - Topic overlap: 0.12 → {CFG.get('acceptance_topic', {}).get('after_text_min', 0.05)}")

print("\n2. Mapping amélioré:")
print(f"   - {len(enhanced_mapping)} entrées avec correspondance parfaite")
print(f"   - {enhanced_mapping['has_text'].mean()*100:.1f}% avec texte")
print(f"   - {enhanced_mapping['enough_length'].mean()*100:.1f}% avec texte suffisant")

print("\n3. Critères assouplies:")
print("   - MODERATE accepté avec garde-fous")
print("   - CRITICAL accepté si amélioration significative")
print("   - Re-summarize moins restrictif")

print("\n📈 IMPACT ATTENDU:")
print(f"   - +{len(candidates_new) - len(old_candidates)} candidats traités")
print(f"   - +{len(new_sufficient) - len(old_sufficient)} textes utilisables pour re-summarize")
print("   - Taux d'acceptation amélioré (à valider)")

print("\n🎯 PRÊT POUR TESTS:")
print("   - Fichiers: level3_enhanced_mapping.csv, level3_text_mapping.csv")
print("   - Config: level3.yaml (mise à jour)")
print("   - Utils: level3_utils.py (fonctions corrigées)")